# The Parity Problem
You are given a function as a black box: $ f: \{0, 1\}^n \rightarrow \{0, 1\}$ and $f(x) = u.x \mod2$.

For some hidden $u \in \{0, 1\}^n$, find $u$ with as few queries as possible. This is a restricted version of the Deutsch–Jozsa algorithm where instead of distinguishing between two different classes of functions, it tries to learn a string encoded in a function. 

## A classical solution

![Classical Solution](/Users/jacksonlu/Downloads/bv.PNG "A classical solution the parity problem")

We need $n$ queries to compute the hidden string, because each time you run this circuit, you get only one bit of information. 
For example: let the hidden string $u = 101 $.

$$ f(100) = 1 $$ 

$$ f(010) = 0 $$

$$ f(001) = 1 $$

One possible solution to implement this classically is to use a classical AND gate as a mask to get each bit of information. 

## The Bernstein-Vazirani Algorithm
In contrast to the classical solution, the quantum solution requires only one query to find the hidden $u$. This algorithm was invented by Ethan Bernstein and Umesh Vazirani in 1992. Your task is to implement this algorithm in Q#. You can read more about the algorithm [here](https://en.wikipedia.org/wiki/Bernstein%E2%80%93Vazirani_algorithm).
![Quantum Solution](/Users/jacksonlu/Downloads/bv-q.PNG "A quantum solution the parity problem")

The Bernstein-Vazirani algorithm is described as follows:
1. Initialize n query qubits in the state $|0\rangle$ and one answer qubit in the state $|-\rangle$
2. Apply the Hadamard Transform to an n-qubit query state to get $H|0\rangle^{\otimes n} =  \frac{1}{2^{n/2}} \sum_x |x> $.
3. Apply the oracle $U_f$ which transforms the state to  $\frac{1}{2^{n/2}} \sum_x (-1)^{u.x} |x> $
4. Apply the Hadamard Transform to the query state after the oracle is applied.
5. Measure in the computational basis.

In [1]:
import pennylane as qml

## Task 1
**Summary**

Prepare qubits in a uniform superposition. For this, use [qml.broadcast](https://docs.pennylane.ai/en/stable/code/api/pennylane.broadcast.html), instead of a loop. This allows us to apply a unitary multiple times to a specific pattern of wires.

**Input**

*query_wires*: an array of numbers representing the query wires

Apply the hadamard transform the query_wires. 


In [15]:
def walshHadamardTransform(query_wires):
    #Apply hadmarad gate to all qubits on query_wire use qml.broadcast
    qml.broadcast(qml.Hadamard, wires=query_wires, pattern="single")
    return query_wires
    

## Task 2
**Summary**

Prepare the answer wire (traditionally, the last qubit on our device) in the state $|-\rangle$

**Input**

*answer_wire*: the answer qubit

In [16]:
def prepareAnswerQubit(answer_wire):
    #put answer_wire in ket1 state 
    qml.PauliX(wires=answer_wire)
    #apply hadamard gate to answer_wire to put it in superposition
    qml.Hadamard(wires=answer_wire)
    return answer_wire
    

## Task 3

**Summary**

Encode the secret string (supplied as a boolean array s) into a function $U_f$

**Input**: 
query_wires: qubit array
answer_wire: answer qubit
secret: secret string as a boolean array e.g. [True, False, True, True]


In [17]:
def Uf(query_wires, answer_wire, secret):
    #secret string as a boolean array e.g. [True, False, True, True]
    for i, val in enumerate(secret):
        if val:
            qml.CNOT(wires=[query_wires[i], answer_wire])
    return query_wires, answer_wire

## Task 4: Explain what gates you applied to create the encoded string as $U_f$ and why you applied them.

You only need to apply CNOT gate, control is any of the query_wires and target is answer_wire. The reason is to introduce phase-kickback at the query_wires.

## Task 5: Run the Bernstein-Vazirani Algorithm using operations defined above

1. Create the secret string, and the required qubits
1. Perform the Bernstein Vazirani algorithm by using the tasks above. 
1. Return probabilities on the query_wires.


In [67]:
# Change the secret string to encode a string of your choice
secret_string = [True, False, False, True]

query_wires = range(len(secret_string))
answer_wire = len(secret_string)

dev = qml.device('default.qubit', wires=(len(secret_string) + 1), shots=1024) # + 1 for the answer wire

In [68]:
@qml.qnode(dev)
def BernsteinVazirani(query_wires, answer_wire, secret_string):
    walshHadamardTransform(query_wires)
    prepareAnswerQubit(answer_wire)
    Uf(query_wires, answer_wire, secret_string)
    walshHadamardTransform(query_wires)
    #measure the query wires only
    return qml.counts(wires=query_wires)   

### Check whether your answer corresponds to the secret string

In [71]:

BernsteinVazirani(query_wires, answer_wire, secret_string)
#print(qml.draw(BernsteinVazirani)(query_wires, answer_wire, secret_string))


{'1001': tensor(1024, requires_grad=True)}